In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install gensim

In [ ]:
from tensorflow.keras.models import load_model
lstm = load_model('data/LSTM_model.h5')

In [ ]:
import re
from googletrans import Translator

def trans(review):
    translator = Translator()
    text = translator.translate(text=review, src="ja", dest="en").text
    text = re.sub(r"[,.!?]", " ", text)
    text = text.lower()
    return [text]

In [ ]:
# word2vecモデル読み込み
import gensim
wv = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)

length = 100
size = 1

# word2vecでembedding
def embedding(seq, wv, length, size):
    res = np.zeros((size, length, 300))
    for i in range(size):
        words = seq[i].split(' ')
        for j in range(len(words)):
            try:
                res[i, j, :] = wv[words[j]]
            except:
                pass
    return res

In [ ]:
def calc(star, review):
    review_en = trans(review)
    embed = embedding(review_en, wv, length, size)
    pred = lstm.predict(embed)[0][0]
    score = star + (pred-0.5)*5*max(0.5, min(1, len(review)/30))
    score = np.round(score, 2)
    score = max(0, score)
    return score

In [ ]:
star = int(input('★の数：'))
review = input('レビュー：').rstrip()
score = calc(star, review)
print('予測スコア：', score)